<a href="https://colab.research.google.com/github/philliewright/Hatespeech_CW2/blob/main/utils_hs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%%writefile "/content/drive/My Drive/Projects_Portfolio/hatespeech_detection/utils_hs.py"

import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

def load_csv(file_path):
    try:
        data = pd.read_csv(file_path)
        if data.empty:
            raise ValueError("The CSV file is empty")
        return data
    except FileNotFoundError:
        raise FileNotFoundError("The specified file does not exist")
    except Exception as e:
        raise Exception(f"An unknown error occurred: {e}")

def report_missing_values(data):
    """ Checks for missing values"""
    missing_values = data.isnull().sum()
    print("Missing values in each column:")
    for column, missing_value_count in missing_values.items():
        print(f"{column}: {missing_value_count}")


Overwriting /content/drive/My Drive/Projects_Portfolio/hatespeech_detection/utils_hs.py


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
